In [1]:
import math
import datetime
import numpy as np
import pickle

import methodsMLinterns
import customAutoencoder

Using Theano backend.


In [2]:
stocks = ['DNB', 'NRG', 'CL', 'ANTM', 'NEE', 'PAYX', 'VAR', 'NI', 'MNST', 'JNJ', 'TGNA', 'NOV', 'FIS', 'BLK', 'HBI', 'NVDA', 'DLTR', 'MRO', 'EMN', 'AMT', 'FLR', 'IBM', 'BK', 'NFX', 'AGN', 'LRCX', 'DIS', 'LH', 'C', 'MNK']

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [4]:
features_1p4_extra_a_z =['aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'abj', 'abm', 'abn', 'abo', 'abp', 'abq', 'abr', 'abs', 'abt', 'abu', 'abv', 'abw', 'abx', 'aby', 'abz', 'aca', 'acb', 'acc', 'acd', 'ace', 'acf', 'acr', 'acw', 'acx', 'acy', 'adi', 'adj', 'adl', 'ado', 'adp', 'adq', 'adr', 'ads', 'adt', 'adu', 'adv', 'adw', 'adx', 'ady', 'adz', 'aea', 'aeb', 'aec', 'aed', 'aee', 'aef', 'aeg', 'aeh', 'aei', 'aej', 'aek', 'ael', 'aem', 'aen', 'aeo', 'aep', 'aeq', 'aer', 'aes', 'aex', 'aey', 'aez', 'afa', 'afj', 'afl', 'afo', 'afp', 'afq', 'afr', 'afs', 'aft', 'afu', 'afv', 'afw', 'afx', 'afy', 'afz', 'aga', 'agb', 'agc', 'agd', 'age', 'agf', 'agg', 'agh', 'agi', 'agj', 'agk', 'agl', 'agm', 'agn', 'ago', 'agp', 'agq', 'agr', 'ags', 'agt', 'agu', 'agv', 'agw', 'agx', 'agy', 'ahf', 'ahg', 'ahh', 'ahi', 'ahj', 'ahk', 'ahl', 'ahm', 'ahn', 'aho', 'zhq', 'zhr', 'zhs', 'zht', 'zhu', 'zhv', 'zhw', 'ziy', 'zjb', 'zjc', 'zjd', 'zje', 'zjf', 'zjg', 'zjh', 'zji', 'zjj', 'zjk', 'zjl', 'zjm', 'zjn', 'zjo', 'zjp', 'zjq', 'zjr', 'zjs', 'zjt', 'zju', 'zkg', 'zkl', 'zkm', 'zkn', 'zkx', 'zky', 'zla', 'zld', 'zle', 'zlf', 'zlg', 'zlh', 'zli', 'zlj', 'zlk', 'zll', 'zlm', 'zln', 'zlo', 'zlp', 'zlq', 'zlr', 'zls', 'zlt', 'zlu', 'zlv', 'zlw', 'zlx', 'zly', 'zlz', 'zma', 'zmb', 'zmc', 'zmd', 'zme', 'zmf', 'zmg', 'zmh', 'zmm', 'zmn', 'zmo', 'zmp', 'zmy', 'zna', 'znd', 'zne', 'znf', 'zng', 'znh', 'zni', 'znj', 'znk', 'znl', 'znm', 'znn', 'zno', 'znp', 'znq', 'znr', 'zns', 'znt', 'znu', 'znv', 'znw', 'znx', 'zny', 'znz', 'zoa', 'zob', 'zoc', 'zod', 'zoe', 'zof', 'zog', 'zoh', 'zoi', 'zoj', 'zok', 'zol', 'zom', 'zon', 'zou', 'zov', 'zow', 'zox', 'zoy', 'zoz', 'zpa', 'zpb', 'zpc', 'zpd', 'zpe']
features_1p4_extra_a=[f for f in features_1p4_extra_a_z if f[0]=='a']
features_1p4_extra_z=[f for f in features_1p4_extra_a_z if f[0]=='z']
assert(len(features_1p4_extra_a)+len(features_1p4_extra_z)==len(features_1p4_extra_a_z))

In [5]:
date_test_set = datetime.date(2016, 1, 1)

clf_portfolio_dic = methodsMLinterns.ClassificationPortfolio(stocks=stocks, minutes_forward=30)
clf_portfolio_dic.loadData()
clf_portfolio_dic.cleanUpData(features_1p4_extra_a_z)
clf_portfolio_dic.getTrainTestSetDate(date_test_set)

### Group together all the stocks

In [6]:
def prepareData(features):
    X_train = np.array([], dtype=np.float64).reshape(0,len(features))
    y_train = np.array([], dtype=np.float64).reshape(0,1)
    X_test = np.array([], dtype=np.float64).reshape(0,len(features))
    y_test = np.array([], dtype=np.float64).reshape(0,1)


    for k, stock in enumerate(clf_portfolio_dic.stocks):
        name = stock + str(clf_portfolio_dic.minutes_forward)
        if k==0:
            X_train, y_train = clf_portfolio_dic.X_train_dic[name][features].as_matrix(),(clf_portfolio_dic.y_train_dic[name]+1)/2
            X_test, y_test = clf_portfolio_dic.X_test_dic[name][features].as_matrix(), (clf_portfolio_dic.y_test_dic[name]+1)/2
        else:
            X_train = np.concatenate((X_train,clf_portfolio_dic.X_train_dic[name][features].as_matrix()),axis=0)
            y_train = np.concatenate((y_train,(clf_portfolio_dic.y_train_dic[name]+1)/2),axis=0)
            X_test = np.concatenate((X_test,clf_portfolio_dic.X_test_dic[name][features].as_matrix()),axis=0)
            y_test = np.concatenate((y_test,(clf_portfolio_dic.y_test_dic[name]+1)/2),axis=0)

    print(X_train.shape)
    # Transform to one hot vectors
    y_t = np.zeros((y_train.shape[0], 2))
    y_t[np.arange(y_train.shape[0]), y_train.astype('int32')] = 1
    y_train = y_t

    y_t = np.zeros((y_test.shape[0], 2))
    y_t[np.arange(y_test.shape[0]), y_test.astype('int32')] = 1
    y_test = y_t
    
    return X_train,y_train,X_test,y_test

### Random Grid Search

Create Train and Validation sets (60% and 40%)

In [7]:
weightsFolder = "weights_RandGrid_run2"
with open("%s/listOfParameters_RandGrid.p"%weightsFolder,'rb') as f:
    listOfParameters, validation_idx = pickle.load(f)

In [8]:
X_train_a, y_train_a, X_test_a, y_test_a = prepareData(features_1p4_extra_a)
X_train_z, y_train_z, X_test_z, y_test_z = prepareData(features_1p4_extra_z)
X_train_a_z, y_train_a_z, X_test_a_z, y_test_a_z = prepareData(features_1p4_extra_a_z)

mask = np.array([(i in validation_idx) for i in range(y_train_a.shape[0])])

X_train_a, y_train_a, X_val_a, y_val_a = X_train_a[~mask,:], y_train_a[~mask,:], X_train_a[mask,:], y_train_a[mask,:]
X_train_z, y_train_z, X_val_z, y_val_z = X_train_z[~mask,:], y_train_z[~mask,:], X_train_z[mask,:], y_train_z[mask,:]
X_train_a_z, y_train_a_z, X_val_a_z, y_val_a_z = X_train_a_z[~mask,:], y_train_a_z[~mask,:], X_train_a_z[mask,:], y_train_a_z[mask,:]

(122607, 119)
(122607, 120)
(122607, 239)


### Test on stocks separately

In [9]:
def testOnSeparateStocks(autoList, feat):
    n = len(autoList)
    trainNumOfTrades = np.zeros((clf_portfolio_dic.N_stocks,))
    valNumOfTrades = np.zeros((clf_portfolio_dic.N_stocks,))
    testNumOfTrades = np.zeros((clf_portfolio_dic.N_stocks,))
    acc_train_autoencoder = np.zeros((n,clf_portfolio_dic.N_stocks,))
    acc_val_autoencoder = np.zeros((n,clf_portfolio_dic.N_stocks,))
    acc_test_autoencoder = np.zeros((n,clf_portfolio_dic.N_stocks,))

    # difficulty is to recreate the validation set for each stock
    indexCount = 0
    for k, stock in enumerate(clf_portfolio_dic.stocks):
        name = "%s30"%stock
        print(k,name)
        
        # extract stock data
        x_train = np.array(clf_portfolio_dic.X_train_dic[name][feat])
        x_test = np.array(clf_portfolio_dic.X_test_dic[name][feat])
        y_train = (clf_portfolio_dic.y_train_dic[name]+1)/2
        y_test = (clf_portfolio_dic.y_test_dic[name]+1)/2
        
        # split train into val+train
        n = y_train.shape[0]
        mask = np.array([(i+indexCount in validation_idx) for i in range(n)])
        x_train, y_train, x_val, y_val = x_train[~mask,:], y_train[~mask], x_train[mask,:], y_train[mask]
        indexCount += n
        assert(n==y_train.shape[0]+y_val.shape[0])

        
        # transform ys to one-hot vectors
        y_t = np.zeros((y_train.shape[0], 2))
        y_t[np.arange(y_train.shape[0]), y_train.astype('int32')] = 1
        y_train = y_t

        y_t = np.zeros((y_val.shape[0], 2))
        y_t[np.arange(y_val.shape[0]), y_val.astype('int32')] = 1
        y_val = y_t
        
        y_t = np.zeros((y_test.shape[0], 2))
        y_t[np.arange(y_test.shape[0]), y_test.astype('int32')] = 1
        y_test = y_t
        
        # store numbers of trades
        trainNumOfTrades[k] = y_train.shape[0]
        valNumOfTrades[k] = y_val.shape[0]
        testNumOfTrades[k] = y_test.shape[0]
    
        # compute accuracies
        for i, a in enumerate(autoList):
            y_pred_train = a.predict(x_train)
            y_pred_val = a.predict(x_val)
            y_pred_test = a.predict(x_test)
            acc_train = (100 * (np.argmax(y_pred_train,1) == np.argmax(y_train,1))).mean()
            acc_val = (100 * (np.argmax(y_pred_val,1) == np.argmax(y_val,1))).mean()
            acc_test = (100 * (np.argmax(y_pred_test,1) == np.argmax(y_test,1))).mean()
            acc_train_autoencoder[i,k] = acc_train
            acc_val_autoencoder[i,k] = acc_val
            acc_test_autoencoder[i,k] = acc_test
        
    return(acc_train_autoencoder, acc_val_autoencoder, acc_test_autoencoder,
           trainNumOfTrades, valNumOfTrades, testNumOfTrades)

In [10]:
expList = []
autoList = []

for k, features in enumerate([features_1p4_extra_a_z]): #features_1p4_extra_a, features_1p4_extra_z
    for j,encoding_size in enumerate([30]): #,50,70
        architecture = [len(features),100,encoding_size]
        if features == features_1p4_extra_a:
            feat = "a"
        elif features == features_1p4_extra_z:
            feat = "z"
        elif features == features_1p4_extra_a_z:
            feat = "a_z"
        lp = listOfParameters["%s_%s"%(feat,encoding_size)]
        
        # for each run
        for i in range(0,lp[0].shape[0]):
            for f in [0,1]:
                d=lp[0][i]
                l2=lp[1][i]
                batchSize=lp[2][i]
                inputNoise=lp[3][i]
                loss= "cp" if lp[4][i] == 'cosine_proximity' else "mse"
                name = "A30S_(%s,%s,%s)_d%s_in%s_r(%s,%s)_b%s_l(%s)_f%s"%(len(features),100, encoding_size,
                                                                      d, inputNoise, 0, l2, batchSize, loss, f)
                print(name)
                e = customAutoencoder.ExperimentPerformance(
                    methodName=name, 
                    stocks=stocks, originalFeatures=features)
                auto = customAutoencoder.Autoencoder([len(features), 100, encoding_size],dropout=d,
                                                     inputNoise=inputNoise,l2reg=l2,autoencoderLoss=lp[4][i])
                auto.buildAutoencoder()
                if f == 1:
                    auto.freezeAutoencoder(True)
                chkpt_0 = "%s/%s.hdf5"%(weightsFolder,name)
                auto.loadFromWeights(chkpt_0)
                
                autoList.append(auto)
                expList.append(e)

        # Train and Test accuracy on separate stocks        
        acc_train_vec, acc_val_vec, acc_test_vec, trainNumOfTrades, valNumOfTrades, testNumOfTrades = testOnSeparateStocks(autoList,features)    
        for i, e in enumerate(expList):
            e.setTrainResults(acc_train_vec[i,:], trainNumOfTrades)
            e.setValResults(acc_val_vec[i,:], valNumOfTrades)
            e.setTestResults(acc_test_vec[i,:], testNumOfTrades)
            print("%s | %s"%(e.getAccuracy(set='train'),e.getAccuracyMinusSigma(set='train')))
            print("%s | %s"%(e.getAccuracy(set='val'),e.getAccuracyMinusSigma(set='val')))
            print("%s | %s"%(e.getAccuracy(set='test'),e.getAccuracyMinusSigma(set='test')))

                

A30S_(239,100,30)_d0.1_in0.0_r(0,0.005)_b128_l(cp)_f0
A30S_(239,100,30)_d0.1_in0.0_r(0,0.005)_b128_l(cp)_f1
A30S_(239,100,30)_d0.1_in0.0_r(0,0.005)_b128_l(mse)_f0
A30S_(239,100,30)_d0.1_in0.0_r(0,0.005)_b128_l(mse)_f1
A30S_(239,100,30)_d0.1_in0.05_r(0,0.005)_b128_l(cp)_f0
A30S_(239,100,30)_d0.1_in0.05_r(0,0.005)_b128_l(cp)_f1
A30S_(239,100,30)_d0.1_in0.1_r(0,0.005)_b128_l(mse)_f0
A30S_(239,100,30)_d0.1_in0.1_r(0,0.005)_b128_l(mse)_f1
A30S_(239,100,30)_d0.1_in0.0_r(0,0.005)_b256_l(mse)_f0
A30S_(239,100,30)_d0.1_in0.0_r(0,0.005)_b256_l(mse)_f1
A30S_(239,100,30)_d0.1_in0.1_r(0,0.005)_b256_l(cp)_f0
A30S_(239,100,30)_d0.1_in0.1_r(0,0.005)_b256_l(cp)_f1
A30S_(239,100,30)_d0.1_in0.1_r(0,0.005)_b256_l(mse)_f0
A30S_(239,100,30)_d0.1_in0.1_r(0,0.005)_b256_l(mse)_f1
A30S_(239,100,30)_d0.1_in0.0_r(0,0.005)_b512_l(cp)_f0
A30S_(239,100,30)_d0.1_in0.0_r(0,0.005)_b512_l(cp)_f1
A30S_(239,100,30)_d0.1_in0.05_r(0,0.005)_b512_l(cp)_f0
A30S_(239,100,30)_d0.1_in0.05_r(0,0.005)_b512_l(cp)_f1
A30S_(239,100,30

A30S_(239,100,30)_d0.5_in0.05_r(0,0.02)_b128_l(cp)_f0
A30S_(239,100,30)_d0.5_in0.05_r(0,0.02)_b128_l(cp)_f1
A30S_(239,100,30)_d0.5_in0.05_r(0,0.02)_b128_l(mse)_f0
A30S_(239,100,30)_d0.5_in0.05_r(0,0.02)_b128_l(mse)_f1
A30S_(239,100,30)_d0.5_in0.1_r(0,0.02)_b128_l(cp)_f0
A30S_(239,100,30)_d0.5_in0.1_r(0,0.02)_b128_l(cp)_f1
A30S_(239,100,30)_d0.5_in0.0_r(0,0.02)_b256_l(cp)_f0
A30S_(239,100,30)_d0.5_in0.0_r(0,0.02)_b256_l(cp)_f1
A30S_(239,100,30)_d0.5_in0.1_r(0,0.02)_b256_l(mse)_f0
A30S_(239,100,30)_d0.5_in0.1_r(0,0.02)_b256_l(mse)_f1
A30S_(239,100,30)_d0.5_in0.05_r(0,0.02)_b512_l(cp)_f0
A30S_(239,100,30)_d0.5_in0.05_r(0,0.02)_b512_l(cp)_f1
A30S_(239,100,30)_d0.5_in0.05_r(0,0.02)_b512_l(mse)_f0
A30S_(239,100,30)_d0.5_in0.05_r(0,0.02)_b512_l(mse)_f1
A30S_(239,100,30)_d0.5_in0.1_r(0,0.02)_b512_l(mse)_f0
A30S_(239,100,30)_d0.5_in0.1_r(0,0.02)_b512_l(mse)_f1
A30S_(239,100,30)_d0.5_in0.0_r(0,0.08)_b128_l(cp)_f0
A30S_(239,100,30)_d0.5_in0.0_r(0,0.08)_b128_l(cp)_f1
A30S_(239,100,30)_d0.5_in0.0_r

KeyboardInterrupt: 

In [ ]:
print(len(expList))

In [12]:
with open("pickles/acc_autoencoder_logit_randGridSearch_run2.p",'wb') as f:
    pickle.dump( expList, f, protocol=pickle.HIGHEST_PROTOCOL)

## Hyperas
### You must create the train/val/test datasets above using validation_idx

In [10]:
import glob
weightsFolder = "weights_End2End"

modelsList = glob.glob("%s/test*.hdf5"%weightsFolder)

#with open("%s/hyperasSearchBestParameters.p"%weightsFolder,'rb') as f:
#    bestParameters = pickle.load(f)

with open("%s/hyperasSearchParameters.p"%weightsFolder,'rb') as f:
    parameters, val_loss1, val_loss2 = pickle.load(f)

In [11]:
if np.isnan(val_loss1).any() or np.isnan(val_loss2).any():
    print("Some models were not properly defined")
    print(val_loss1)
    print(val_loss2)
else:
    print("No model produced NaN losses")

Some models were not properly defined
[0.33968847180075884, 0.35651177585164717, 0.37161706411072148, 0.33782876402374468, 0.33962792399996056, 0.34011151076037388, 0.35343391805303526, 0.36985242011053104, 0.33864634530958065, 0.36939604498442125, 0.33839789373878965, 0.33746986597288403, 0.34033723624779721, 0.33821592535866368, 0.33790840380069148, 0.3435826678444649, 0.33724209108024439, nan, 0.37027361689665811, 0.33974626065831542, 0.35309264889326131, 0.33932463650634065, 0.35809449071841642, 0.33721673223120197, 0.36748837710263044, 0.33747000167285579, 0.39445090700778096, 0.35649805116195321, 0.33753097629702578, 0.33835424092158717, 0.33828003642608456, 0.3387504945609171, 0.39698531517010532, 0.33792771039419167, 0.33937130962613749, 0.33713865885339678, 0.33864479415202792, 0.33651638250057236, 0.33775702708972682, 0.34112385219638958, 0.34177164437845098, nan, 0.33787533307426454, 0.33769362212366838, 0.34010451664206581, 0.33613125900148288, 0.33973532579619331, 0.337757

In [12]:
#print(bestParameters)
print("-------------------------------")
for p in parameters:
    print(p)

-------------------------------
{'batch_size': 256, 'l2': 0.08, 'd': 0.5540312274525752, 'd_1': 0.1702403465020466}
{'batch_size': 256, 'l2': 0.005, 'd': 0.34018289349476505, 'd_1': 0.18652907557981876}
{'batch_size': 256, 'l2': 0.02, 'd': 0.5820465324867875, 'd_1': 0.6102499442616852}
{'batch_size': 128, 'l2': 0.02, 'd': 0.33012965824857, 'd_1': 0.38755641898560955}
{'batch_size': 512, 'l2': 0.005, 'd': 0.2997080033339254, 'd_1': 0.4670652522628885}
{'batch_size': 128, 'l2': 0.08, 'd': 0.5942675273917278, 'd_1': 0.12836945371568076}
{'batch_size': 128, 'l2': 0.005, 'd': 0.6057476433649138, 'd_1': 0.3178435226292249}
{'batch_size': 512, 'l2': 0.08, 'd': 0.06462645089021866, 'd_1': 0.6584858720429755}
{'batch_size': 512, 'l2': 0.005, 'd': 0.5886190500483193, 'd_1': 0.13255605643095045}
{'batch_size': 128, 'l2': 0.02, 'd': 0.1675741976038202, 'd_1': 0.18224033242119786}
{'batch_size': 128, 'l2': 0.02, 'd': 0.4084965623640273, 'd_1': 0.2348196393497661}
{'batch_size': 256, 'l2': 0.02, 'd'

In [13]:
expList = []
autoList = []

for k, features in enumerate([features_1p4_extra_a_z]): #features_1p4_extra_a, features_1p4_extra_z
    for j,encoding_size in enumerate([30]): #,50,70
        architecture = [len(features),100,encoding_size]
        '''
        if features == features_1p4_extra_a:
            feat = "a"
        elif features == features_1p4_extra_z:
            feat = "z"
        elif features == features_1p4_extra_a_z:
            feat = "a_z"
        lp = listOfParameters["%s_%s"%(feat,encoding_size)]
        '''
        
        # for each run
        for i, params in enumerate(parameters):
            for f in [0]: # ,1
                d=np.round(params["d"],decimals=3)
                l2=params["l2"]
                batchSize=params["batch_size"]
                inputNoise=np.round(params["d_1"],decimals=3)
                loss= "cp"# if params["autoencoderLoss"] == 'cosine_proximity' else "mse"
                name = "A30S_(%s,%s,%s)_d%s_in%s_r(%s,%s)_b%s_l(%s)_f%s"%(len(features),100, encoding_size,
                                                                      d, inputNoise, 0, l2, batchSize, loss, f)
                print(name)
                e = customAutoencoder.ExperimentPerformance(
                    methodName=name, 
                    stocks=stocks, originalFeatures=features)
                auto = customAutoencoder.Autoencoder([len(features), 100, encoding_size],dropout=d,
                                                     inputNoise=inputNoise,l2reg=l2,
                                                     autoencoderLoss='cosine_proximity')# params["autoencoderLoss"])
                auto.buildAutoencoder()
                if f == 1:
                    auto.freezeAutoencoder(True)
                chkpt_0 = modelsList[i]
                auto.loadFromWeights(chkpt_0)
                
                autoList.append(auto)
                expList.append(e)

        # Train and Test accuracy on separate stocks        
        acc_train_vec, acc_val_vec, acc_test_vec, trainNumOfTrades, valNumOfTrades, testNumOfTrades = testOnSeparateStocks(autoList,features)    
        for i, e in enumerate(expList):
            e.setTrainResults(acc_train_vec[i,:], trainNumOfTrades)
            e.setValResults(acc_val_vec[i,:], valNumOfTrades)
            e.setTestResults(acc_test_vec[i,:], testNumOfTrades)
            print("%s | %s"%(e.getAccuracy(set='train'),e.getAccuracyMinusSigma(set='train')))
            print("%s | %s"%(e.getAccuracy(set='val'),e.getAccuracyMinusSigma(set='val')))
            print("%s | %s"%(e.getAccuracy(set='test'),e.getAccuracyMinusSigma(set='test')))


A30S_(239,100,30)_d0.554_in0.17_r(0,0.08)_b256_l(cp)_f0
A30S_(239,100,30)_d0.34_in0.187_r(0,0.005)_b256_l(cp)_f0
A30S_(239,100,30)_d0.582_in0.61_r(0,0.02)_b256_l(cp)_f0
A30S_(239,100,30)_d0.33_in0.388_r(0,0.02)_b128_l(cp)_f0
A30S_(239,100,30)_d0.3_in0.467_r(0,0.005)_b512_l(cp)_f0
A30S_(239,100,30)_d0.594_in0.128_r(0,0.08)_b128_l(cp)_f0
A30S_(239,100,30)_d0.606_in0.318_r(0,0.005)_b128_l(cp)_f0
A30S_(239,100,30)_d0.065_in0.658_r(0,0.08)_b512_l(cp)_f0
A30S_(239,100,30)_d0.589_in0.133_r(0,0.005)_b512_l(cp)_f0
A30S_(239,100,30)_d0.168_in0.182_r(0,0.02)_b128_l(cp)_f0
A30S_(239,100,30)_d0.408_in0.235_r(0,0.02)_b128_l(cp)_f0
A30S_(239,100,30)_d0.244_in0.017_r(0,0.02)_b256_l(cp)_f0
A30S_(239,100,30)_d0.301_in0.564_r(0,0.02)_b512_l(cp)_f0
A30S_(239,100,30)_d0.414_in0.021_r(0,0.005)_b512_l(cp)_f0
A30S_(239,100,30)_d0.418_in0.285_r(0,0.02)_b256_l(cp)_f0
A30S_(239,100,30)_d0.558_in0.778_r(0,0.02)_b256_l(cp)_f0
A30S_(239,100,30)_d0.53_in0.146_r(0,0.02)_b128_l(cp)_f0
A30S_(239,100,30)_d0.443_in0.654_

In [14]:
print(len(expList))

50


In [15]:
with open("pickles/acc_autoencoder_logit_End2EndHyperas_run2.p",'wb') as f:
    pickle.dump( expList, f, protocol=pickle.HIGHEST_PROTOCOL)